In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
from bokeh.plotting import output_notebook
output_notebook()

# Hovering

Bokeh has simple hovering to make it easy to drill down on your data:


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.sampledata.iris import flowers

colormap = {'setosa': 'red', 'versicolor': 'green', 'virginica': 'blue'}
colors = [colormap[x] for x in flowers['species']]

tools='hover,pan,box_zoom,save'

p = figure(title = "Iris Morphology", tools=tools)
p.xaxis.axis_label = 'Petal Length'
p.yaxis.axis_label = 'Petal Width'

p.circle(flowers["petal_length"], flowers["petal_width"],
         color=colors, fill_alpha=0.2, size=10)

output_notebook()
show(p)

By default, Bokeh uses the *index* of the input dataframe in its hover panel.

## Exercise

- Load the cars dataset
- Add a column, "car year" that contains both the car model and the year. 
- Set that column as the index
- then use the Hover tool as above to show car mpg vs weight.


# Custom hover tips

Rather than setting the index, Bokeh lets you customize the hover tooltip itself, but it's a bit more verbose. On the plus side, any valid html [can be a tooltip](https://github.com/jni/blob-explorer/blob/bd9fa676a2a23317e2ea84bdf48b19e71b9e75d4/picker.py#L120), which means that you can actually embed images in the tooltip. This is great for exploring image data. (Though we won't see this here.)

In [ ]:
from bokeh.models import HoverTool

# make sure color is its own column
flowers['colors'] = colors

tools='hover,pan,box_zoom,save'

p = figure(title = "Iris Morphology", tools=tools)
p.xaxis.axis_label = 'Petal Length'
p.yaxis.axis_label = 'Petal Width'

p.circle('petal_width', 'petal_length', source=flowers,
         color='colors', fill_alpha=0.2, size=10)

# grab the hover tool and change the tooltip
p.select_one(HoverTool).tooltips = [
    #("tooltip name", "@column name"),
    ("Species", "@species"),
    ("Sepal length", "@sepal_length"),
    ("Sepal width", "@sepal_width"),
    ("Petal length", "@petal_length"),
    ("Petal width", "@petal_width"),
]

show(p) 

# Linked brushing

Sometimes the identity of points in different points can be traced by *linked brushing*, selecting a subset of data between different plots. Bokeh does this transparently as long as different figures share the same data source:

In [ ]:
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

x = list(range(-20, 21))
y0 = [abs(xx) for xx in x]
y1 = [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "pan,box_select,lasso_select,reset,save"

# create a new plot and add a renderer
left = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
right.circle('x', 'y1', source=source)

p = gridplot([[left, right]])

show(p)

## Exercise

- Draw two scatterplots from the cars dataset, using linked brushing.
- Repeat the asteroids visualization using Bokeh hover (show the asteroid "moID" at least), and using linked brushing. You may or may not want to set the background etc.

## Gapminder and JS interactions

Bokeh can build interactions into the browser using just a bit of javascript.

In [ ]:
gapminder = pd.read_csv('data/gapminderDataFiveYear.csv')

In [ ]:
from bokeh.io import show

from bokeh.models import (Text, Plot, Slider, Circle, Range1d,
                          CustomJS, HoverTool, LinearAxis,
                          ColumnDataSource, SingleIntervalTicker)

from bokeh.palettes import Spectral6


In [ ]:
scaling  = 200
gapminder['popsize'] = np.sqrt(gapminder['pop'] /
                               np.pi) / scaling
min_size = 3
gapminder['popsize'] = gapminder['popsize'].where(
              gapminder['popsize'] >= min_size, other=np.nan
              ).fillna(min_size)


In [ ]:
region_list = sorted(set(gapminder['continent']))
region_dict = dict(zip(region_list, range(len(region_list))))

def get_color(r):
    return Spectral6[region_dict[r]]

gapminder['regcolor'] = gapminder['continent'].apply(get_color)

In [ ]:
gapminder['loggdpcap'] = np.log10(gapminder['gdpPercap'] + 1)

In [ ]:
years = sorted(set(gapminder['year']))

sources = {}

for year, table in gapminder.groupby('year'):
    sources[f'_{year}'] = ColumnDataSource(table)

In [ ]:
xdr = Range1d(0.95 * np.min(gapminder['loggdpcap']),
              1.05 * np.max(gapminder['loggdpcap']))
ydr = Range1d(0.99 * np.min(gapminder['lifeExp']),
              1.05 * np.max(gapminder['lifeExp']))

plot = Plot(
    x_range=xdr,
    y_range=ydr,
    plot_width=800,
    plot_height=400,
    outline_line_color=None,
    toolbar_location=None,
    min_border=20,
)


In [ ]:
# build axes
xaxis = LinearAxis(
    ticker     = SingleIntervalTicker(interval=1),
    axis_label = "log10(gdp per capita)"
)
yaxis = LinearAxis(
    ticker     = SingleIntervalTicker(interval=20),
    axis_label = "Life expectancy at birth (years)"
)   

plot.add_layout(xaxis, 'below')
plot.add_layout(yaxis, 'left')


In [ ]:
text_source = ColumnDataSource({'year': [f'{years[0]}']})
text        = Text(
                  x=2.5, y=35, text='year',
                  text_font_size='150pt',
                  text_color='#EEEEEE'
                  )
plot.add_glyph(text_source, text);


In [ ]:
renderer_source = sources[f'_{years[0]}']
circle_glyph    = Circle(
                    x='loggdpcap', y='lifeExp',
                    size='popsize', fill_alpha=0.8,
                    fill_color='regcolor',
                    line_color='#7c7e71',
                    line_width=0.5, line_alpha=0.5
                    )

circle_renderer = plot.add_glyph(renderer_source, circle_glyph)


In [ ]:
# Add hover for the circle (not other plot elements)
tooltips = "@country"
plot.add_tools(HoverTool(
                  tooltips=tooltips,
                  renderers=[circle_renderer]
                  )
              )

In [ ]:
# add the legend. x and y are data coordinates.

text_x = 4.5
text_y = 60
for i, region in enumerate(sorted(set(gapminder['continent']))):
    plot.add_glyph(Text(
                      x=text_x, y=text_y,
                      text=[region],
                      text_font_size='10pt',
                      text_color='#666666'
                      )
                  )
    plot.add_glyph(Circle(
                      x=text_x - 0.1,
                      y=text_y + 2,
                      fill_color=Spectral6[i],
                      line_color=None,
                      fill_alpha=0.8,
                      size=10,
                      )
                  )
    text_y -= 5  # move on to next legend text

In [ ]:
# Add the slider. This requires some js munging
dict_of_sources = {x: f'_{x}' for x in years}

js_source_array = str(dict_of_sources).replace("'", "")

code = """
    var year = slider.get('value'),
        sources = %s,
        new_source_data = sources[year].get('data');
    renderer_source.set('data', new_source_data);
    text_source.set('data', {'year': [String(year)]});
""" % js_source_array

callback = CustomJS(args=sources, code=code)
slider   = Slider(
              start=years[0], end=years[-1],
              value=1, step=5, title="Year",
              callback=callback
              )
callback.args["renderer_source"] = renderer_source
callback.args["text_source"] = text_source
callback.args["slider"] = slider


In [ ]:
from bokeh.layouts import layout
show(layout([[plot], [slider]], sizing_mode='scale_width'))